In [1]:
from CustomFunctions import PILRagg
from CustomFunctions import linear_cycle_utils
from aicscytoparam import cytoparam
import re
import vtk
import multiprocessing
import numpy as np
import pandas as pd
import os
from aicsimageio.writers.ome_tiff_writer import OmeTiffWriter

def collect_results(result):
    """Uses apply_async's callback to setup up a separate Queue for each process.
    This will allow us to collect the results from different threads."""
    results.append(result)

In [2]:
#get directories and open separated datasets

time_interval = 5 #sec/frame

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'random/'
if not os.path.exists(savedir):
    os.makedirs(savedir)
pilr_fl = datadir + 'PILRs/'
pilragg_fl = savedir + 'avgPILRs/'
if not os.path.exists(pilragg_fl):
    os.makedirs(pilragg_fl)
    
binframe = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
binframe = binframe[['cell']+[x for x in binframe.columns.to_list() if 'bin' in x]]

TotalFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)
TotalFrame = TotalFrame.drop(columns = [x for x in TotalFrame.columns.to_list() if 'bins' in x])
TotalFrame = TotalFrame.merge(binframe, right_on = 'cell', left_on = 'cell')

centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)

nbins = np.max(TotalFrame[[x for x in TotalFrame.columns.to_list() if 'bin' in x]].to_numpy())

#restrict dataframe to only random experiments
df = TotalFrame[TotalFrame.Treatment=='Random'].copy()

In [3]:
############## Average PILRs of entire dataset and split by migration mode #########################    
#specify number of PCs and number of bins
npcs = 10
nbins = 5
bins = list(range(1,nbins+1))
nisos = [1,63]
lmax = 10

#make tiny sphere for inner mesh
#create inner sphere
sphereSource = vtk.vtkSphereSource()
sphereSource.SetCenter(0.0, 0.0, 0.0)
sphereSource.SetRadius(nisos[0]/2)
# Make the surface smooth.
sphereSource.SetPhiResolution(100)
sphereSource.SetThetaResolution(100)
sphereSource.Update()
spherepoly = sphereSource.GetOutput()





pcdf = df[[x for x in df.columns.to_list() if 'bins' in x]]
for g in df.structure.unique():
    for i, p in enumerate(pcdf.columns.to_list()):
        for b in bins:
            allpilrs = []

            temp = pcdf[p][df.structure==g]
            num = temp[temp==b].cell.to_list()
            if __name__ ==  '__main__':
                # use multiprocessing to perform segmentation and x,y,z determination
                pool = multiprocessing.Pool(processes=60)
                results= []
                for n in num:
                    pool.apply_async(PILRagg.read_parameterized_intensity, args=(pilr_fl + n + '_PILR.tiff',), 
                                     callback=collect_results)
                pool.close()
                pool.join()
            pagg = np.array(results)
            allpilrs.append(pagg)
            #get average representation
            pagg_avg = np.mean(pagg, axis = 0)
            #normalize representations
            pagg_norm = PILRagg.normalize_representations(pagg)
            pagg_norm_avg = np.mean(pagg_norm, axis = 0)
            dims = [['X', 'Y', 'Z', 'C', 'T'][d] for d in range(pagg_avg.ndim)]
            dims = ''.join(dims[::-1])
            OmeTiffWriter.save(pagg_avg, pilragg_fl+p[:-1]+str(b)+f'_{g}_repsagg.tif', dim_order=dims)
            OmeTiffWriter.save(pagg_norm_avg, pilragg_fl+p[:-1]+str(b)+f'_{g}_repsagg_norm.tif', dim_order=dims)


            mesh_outer = PILRagg.read_vtk_polydata(datadir+'PC_Meshes/'+f'Cell_PC{i+1}_{b}_Cell.vtp')
            domain, origin = cytoparam.voxelize_meshes([mesh_outer, spherepoly])
            coords_param, _ = cytoparam.parameterize_image_coordinates(
                seg_mem=(domain>0).astype(np.uint8),
                seg_nuc=(domain>1).astype(np.uint8),
                lmax=lmax,
                nisos=nisos
            )

            morphed = cytoparam.morph_representation_on_shape(
                        img=domain,
                        param_img_coords=coords_param,
                        representation=pagg_avg)
            morphed = np.stack([domain, morphed])
            OmeTiffWriter.save(morphed, pilragg_fl+p[:-1]+str(b)+f'_{g}_aggmorph.tif', dim_order='CZYX')

            print(f'Done with {g} {p} bin {b}')

Done with myosin PC1_bins bin 1
Done with myosin PC1_bins bin 2
Done with myosin PC1_bins bin 3
Done with myosin PC1_bins bin 4
Done with myosin PC1_bins bin 5
Done with myosin PC2_bins bin 1
Done with myosin PC2_bins bin 2
Done with myosin PC2_bins bin 3
Done with myosin PC2_bins bin 4
Done with myosin PC2_bins bin 5
Done with myosin PC3_bins bin 1
Done with myosin PC3_bins bin 2
Done with myosin PC3_bins bin 3
Done with myosin PC3_bins bin 4
Done with myosin PC3_bins bin 5
Done with myosin PC4_bins bin 1
Done with myosin PC4_bins bin 2
Done with myosin PC4_bins bin 3
Done with myosin PC4_bins bin 4
Done with myosin PC4_bins bin 5
Done with myosin PC5_bins bin 1
Done with myosin PC5_bins bin 2
Done with myosin PC5_bins bin 3
Done with myosin PC5_bins bin 4
Done with myosin PC5_bins bin 5
Done with myosin PC6_bins bin 1
Done with myosin PC6_bins bin 2
Done with myosin PC6_bins bin 3
Done with myosin PC6_bins bin 4
Done with myosin PC6_bins bin 5
Done with myosin PC7_bins bin 1
Done wit

In [3]:
########### linearize random untreated data and animate them

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'random/'
if not os.path.exists(savedir):
    os.makedirs(savedir)


origin = [5, 7]
whichpcs = [2,7]
binrange = 10
direction = 'counterclockwise'

coeff_df = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)

#### linearize cycle in CGPS
angframe = linear_cycle_utils.linearize_cycle_continuous(
            df,
            centers,
            origin,
            whichpcs,  
            binrange, 
            direction,)

#### get a movie of the average SH coeffs around the cycle
linear_cycle_utils.animate_linear_cycle_shcoeffs(
                        angframe, #pandas dataframe with the CGPS angular coordinates
                        coeff_df, #pandas dataframe with the shcoeffs
                        savedir,
                        whichpcs,
                        binrange,
                        )

#### get the average PILRs around the cycle
if __name__ ==  '__main__':
    linear_cycle_utils.get_linear_cycle_PILRs(
                        angframe, #pandas dataframe with the CGPS angular coordinates
                        savedir,
                        pilr_fl,
                        )

#### combine the PILRs around the cycle into one movie
structurelist=[]
for x in os.listdir(savedir + f'/PC{whichpcs[0]}-PC{whichpcs[1]}_Cycle_AllSHCoeff_Visualization/avgPILRs/'):
    struct = x.split('_')[-2]
    if struct not in structurelist and 'aggmorph' in x: structurelist.append(struct)
for structure in structurelist:
    linear_cycle_utils.combine_linear_PILRs(savedir,
                                            structure,
                                            whichpcs,
                                            )

NameError: name 'df' is not defined

In [4]:
##### load blebbistatin data

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'paranitroblebbistatin/'
if not os.path.exists(savedir):
    os.makedirs(savedir)
pilr_fl = datadir + 'PILRs/'
pilragg_fl = savedir + 'avgPILRs/'
if not os.path.exists(pilragg_fl):
    os.makedirs(pilragg_fl)
    
binframe = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)
binframe = binframe[['cell']+[x for x in binframe.columns.to_list() if 'bin' in x]]

TotalFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)
TotalFrame = TotalFrame.drop(columns = [x for x in TotalFrame.columns.to_list() if 'bins' in x])
TotalFrame = TotalFrame.merge(binframe, right_on = 'cell', left_on = 'cell')

centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)

nbins = np.max(TotalFrame[[x for x in TotalFrame.columns.to_list() if 'bin' in x]].to_numpy())

#restrict dataframe
df = TotalFrame[TotalFrame.Treatment.isin(['DMSO','Para-Nitro-Blebbistatin'])].copy()

In [6]:
########### linearize blebbistatin data and animate them

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'paranitroblebbistatin/'
if not os.path.exists(savedir):
    os.makedirs(savedir)


origin = [5, 7]
whichpcs = [2,7]
binrange = 10
direction = 'counterclockwise'

coeff_df = pd.read_csv(datadir + 'Shape_Metrics_with_Digitized_PCs.csv', index_col=0)

#### linearize cycle in CGPS
angframe = linear_cycle_utils.linearize_cycle_continuous(
            df,
            centers,
            origin,
            whichpcs,  
            binrange, 
            direction,)

#### get a movie of the average SH coeffs around the cycle
linear_cycle_utils.animate_linear_cycle_shcoeffs(
                        angframe, #pandas dataframe with the CGPS angular coordinates
                        coeff_df, #pandas dataframe with the shcoeffs
                        savedir,
                        whichpcs,
                        binrange,
                        )

#### get the average PILRs around the cycle
if __name__ ==  '__main__':
    linear_cycle_utils.get_linear_cycle_PILRs(
                        angframe, #pandas dataframe with the CGPS angular coordinates
                        savedir,
                        pilr_fl,
                        )

#### combine the PILRs around the cycle into one movie
structurelist=[]
for x in os.listdir(savedir + f'/PC{whichpcs[0]}-PC{whichpcs[1]}_Cycle_AllSHCoeff_Visualization/avgPILRs/'):
    struct
    structurelist.append if struct not in structurelist
for st in structurelist:
    linear_cycle_utils.combine_linear_PILRs(savedir,
                                            structure,
                                            )

Finished frame_0_DMSO_actin
Finished frame_10_DMSO_actin
Finished frame_20_DMSO_actin
Finished frame_30_DMSO_actin
Finished frame_40_DMSO_actin
Finished frame_50_DMSO_actin
Finished frame_60_DMSO_actin
Finished frame_70_DMSO_actin
Finished frame_80_DMSO_actin
Finished frame_90_DMSO_actin
Finished frame_100_DMSO_actin
Finished frame_110_DMSO_actin
Finished frame_120_DMSO_actin
Finished frame_130_DMSO_actin
Finished frame_140_DMSO_actin
Finished frame_150_DMSO_actin
Finished frame_160_DMSO_actin
Finished frame_170_DMSO_actin
Finished frame_180_DMSO_actin
Finished frame_190_DMSO_actin
Finished frame_200_DMSO_actin
Finished frame_210_DMSO_actin
Finished frame_220_DMSO_actin
Finished frame_230_DMSO_actin
Finished frame_240_DMSO_actin
Finished frame_250_DMSO_actin
Finished frame_260_DMSO_actin
Finished frame_270_DMSO_actin
Finished frame_280_DMSO_actin
Finished frame_290_DMSO_actin
Finished frame_300_DMSO_actin
Finished frame_310_DMSO_actin
Finished frame_320_DMSO_actin
Finished frame_330_DM